In [2]:
#Aim is to use the description for each company and geenrate a similarity score
#  with the input tags that will help the user to identify there needs properly

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("export.csv")
  


In [5]:
df.head()

,id,ranking,temp_ranking,previous_ranking,company_name,city,country,state,current_employees,last_employees,employee_growth,founded,total_funding,linkedin_url,url,Industry,Points,valuation,valuation_as_of,Contact Data
0,1045228,1,NaN,NaN,Anysphere,San Francisco,United States,CA,577,54,166.67,NaN,$176M,http://www.linkedin.com/company/anysphereinc,anysphere.inc,AI,3002,2.500000e+09,January 2025,For Contacts and mobile phone number see lead411
1,966653,2,NaN,NaN,ElevenLabs,New York,United States,NY,333,115,181.32,2022.0,$353M,http://www.linkedin.com/company/elevenlabsio,elevenlabs.io,AI,2721,3.300000e+09,January 2025,For Contacts and mobile phone number see lead411
2,977455,3,NaN,NaN,Mistral AI,Paris,FRA,NaN,360,119,204.44,2023.0,$1.19B,http://www.linkedin.com/company/mistralai,mistral.ai,AI,2545,6.000000e+09,June 2024,For Contacts and mobile phone number see lead411
3,976147,4,NaN,NaN,Figure,Sunnyvale,United States,CA,370,174,112.82,2022.0,$845M,http://www.linkedin.com/company/figure-ai,figure.ai,Robotics,2384,2.600000e+09,March 2024,For Contacts and mobile phone number see lead411
4,376099,5,NaN,NaN,Abridge,Pittsburgh,United States,PA,326,120,171.67,2018.0,$464M,http://www.linkedin.com/company/abridgehq,abridge.com,AI,2366,5.300000e+09,May 2025,For Contacts and mobile phone number see lead411


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1000 non-null   int64  
 1   ranking            1000 non-null   int64  
 2   temp_ranking       39 non-null     float64
 3   previous_ranking   0 non-null      float64
 4   company_name       1000 non-null   object 
 5   city               929 non-null    object 
 6   country            913 non-null    object 
 7   state              671 non-null    object 
 8   current_employees  1000 non-null   int64  
 9   last_employees     1000 non-null   int64  
 10  employee_growth    1000 non-null   float64
 11  founded            712 non-null    float64
 12  total_funding      558 non-null    object 
 13  linkedin_url       1000 non-null   object 
 14  url                999 non-null    object 
 15  Industry           997 non-null    object 
 16  Points             1000 n

In [7]:
from bs4 import BeautifulSoup
import requests

def get_description(row):
    company_name = row["company_name"]
    url = f'https://growjo.com/company/{company_name}'
    response = requests.get(url)

    if response.ok:
        soup = BeautifulSoup(response.content, 'html.parser')
        p_tag = soup.find('p')
        if p_tag:
            return p_tag.get_text(strip=True)
        else:
            return "Description not found"
    else:
        return "Request failed"


In [8]:
df["description"] = df.apply(get_description,axis=1)

In [9]:

df["description"].iloc[0]
    

"We're Anysphere, the team behind Cursor. \r\n\r\nOur mission is to automate coding. Our approach is to build the engineer of the future: a human-AI programmer that's an order of magnitude more effective than any one programmer.\r\n\r\nThis hybrid engineer will have effortless control over their codebase and no low-entropy keystrokes. They will iterate at the speed of their judgment, even in the most complex systems. Using a combination of AI and human ingenuity, they will out-smart and out-engineer the best pure-AI system."

In [10]:
des=df["description"]

In [11]:
#pre-proccessing of the texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))


import re
from typing import List

# Example stopwords list (expand this as needed)
stop_words = set([
    'the', 'and', 'is', 'in', 'at', 'of', 'a', 'an', 'to', 'with', 'for', 'on'
])

def preprocessor(text: str) -> str:
    clean_text = re.sub(r'[\n\r\t\\]+', ' ', text)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

 
    clean_text = clean_text.lower()

  
    clean_text = re.sub(r'[^\w\s]', '', clean_text)


    words = clean_text.split()

    filtered = [word for word in words if word not in stop_words]

    return " ".join(filtered)


    

In [12]:
df["description"]=df["description"].apply(preprocessor)

In [13]:
df["description"].tolist()

['were anysphere team behind cursor our mission automate coding our approach build engineer future humanai programmer thats order magnitude more effective than any one programmer this hybrid engineer will have effortless control over their codebase no lowentropy keystrokes they will iterate speed their judgment even most complex systems using combination ai human ingenuity they will outsmart outengineer best pureai system',
 'research lab exploring new frontiers voice generation we are researching deploying novel methods voice ai make content enjoyable any language voice benefit new audiences viewers',
 'fast opensource secure language models facilitated specialisation models business usecases leveraging private data usage feedback built from worldclass team europe targeting global market join team',
 'figure ai robotics company building worlds first commercially viable autonomous humanoid robot we are based sunnyvale ca',
 'abridge helps people stay top their health one conversation t

In [14]:
#Feature Extraction
#Here we will append all the columns value that will give us similarity insights
#The Columns will be
#city country state industry description
new_df = df[["company_name","city",'country',"Industry","description"]]

In [15]:

new_df["city"]=new_df["city"].fillna(method="ffill")
new_df["country"]=new_df["country"].fillna(method="ffill")
new_df["Industry"]=new_df["Industry"].fillna(method="ffill")
new_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  1000 non-null   object
 1   city          1000 non-null   object
 2   country       1000 non-null   object
 3   Industry      1000 non-null   object
 4   description   1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


C:\Users\debji\AppData\Local\Temp\ipykernel_19360\3810820409.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_df["city"]=new_df["city"].fillna(method="ffill")
C:\Users\debji\AppData\Local\Temp\ipykernel_19360\3810820409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["city"]=new_df["city"].fillna(method="ffill")
C:\Users\debji\AppData\Local\Temp\ipykernel_19360\3810820409.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_df["country"]=new_df["country"].fillna(method="ffill")
C:\Users\debji\AppData\Local\Temp\ipykernel_19360\3810820409.py:2: Setting

In [16]:
def join_features(row) :

    return " ".join(row[1:].to_list()).lower()


In [17]:
new_df["Details"] = new_df.apply(join_features,axis=1)

C:\Users\debji\AppData\Local\Temp\ipykernel_19360\1951204466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Details"] = new_df.apply(join_features,axis=1)


In [18]:
new_df=new_df.drop(columns=["city","country","Industry","description"])

In [19]:
new_df

,company_name,Details
0,Anysphere,san francisco united states ai were anysphere ...
1,ElevenLabs,new york united states ai research lab explori...
2,Mistral AI,paris fra ai fast opensource secure language m...
3,Figure,sunnyvale united states robotics figure ai rob...
4,Abridge,pittsburgh united states ai abridge helps peop...
...,...,...
995,Renaissance Technologies,east setauket united states investments reques...
996,Electronics and Electrical Design,", in electronics request failed"
997,Oxford Quantum Circuits,reading uk tech services request failed
998,Mosaic Manufacturing,toronto can 3d request failed


In [20]:
#Now we will use Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=1000,binary=True)
y=cv.fit_transform(new_df["Details"]).toarray()

In [21]:

y=y.tolist()

In [22]:
new_df["Vectorised"] = y

In [23]:
new_df.head()

,company_name,Details,Vectorised
0,Anysphere,san francisco united states ai were anysphere ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ElevenLabs,new york united states ai research lab explori...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Mistral AI,paris fra ai fast opensource secure language m...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Figure,sunnyvale united states robotics figure ai rob...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Abridge,pittsburgh united states ai abridge helps peop...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [24]:
new_df.to_csv("Description.csv")

In [25]:
import joblib

# Save model to file
joblib.dump(cv, 'simscore_model.pkl')

['simscore_model.pkl']

In [26]:
#Now we will use cosine similarity to calculate the similarity score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_similarity(tags):
    scores = []

    tag_vector = cv.transform(tags).toarray() 

    for i in range(new_df.shape[0]):
        vector = new_df.iloc[i, -1] 
        vector = np.array(vector).reshape(1, -1)

        # Compute cosine similarity
        score = cosine_similarity(vector, tag_vector)[0][0]
        scores.append(score)

    return scores




In [27]:
x=find_similarity(["ai machine learning"])

In [28]:
x

[np.float64(0.09622504486493763),
 np.float64(0.13245323570650439),
 np.float64(0.13245323570650439),
 np.float64(0.14907119849998599),
 np.float64(0.28097574347450827),
 np.float64(0.0),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.0),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.2182178902359924),
 np.float64(0.25819888974716115),
 np.float64(0.25819888974716115),
 np.float64(0.14907119849998599),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.2182178902359924),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.2357022603955159),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.11111111111111113),
 np.float64(0.0),
 np.float64(0.0),
 np.float6

In [29]:
new_df["Similarity_score"] = x

In [30]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
new_df["Similarity_score"] = np.round(sc.fit_transform(new_df[["Similarity_score"]]),3)

In [31]:
new_df.sort_values(by="Similarity_score",ascending=False).head(20)

,company_name,Details,Vectorised,Similarity_score
202,Nomagic,warsaw pol ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000
4,Abridge,pittsburgh united states ai abridge helps peop...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0.973
469,Taktile,oxford united states fintech businesses are ca...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.970
333,Parloa,berlin deu ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
938,Wayve,london uk ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
19,Synthesia,london uk ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
293,Luminance,london uk ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
125,Cohere,toronto can ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
102,PolyAI,london uk ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894
104,Weka.IO,campbell united states ai request failed,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.894


In [33]:
len(new_df.iloc[0,-2])

1000

1000